# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 01: Backfill Features to the Feature Store</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/bitcoin/1_backfill_feature_groups.ipynb)

## 🗒️ This notebook is divided in 3 sections:
1. Loading the data 
2. Connect to the Hopsworks feature store.
3. Create feature groups and insert them to the feature store.

![tutorial-flow](../images/01_featuregroups.png)

---
## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

## API keys are stored in .env file in the next format:
`BINANCE_API_KEY = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"`

`BINANCE_API_SECRET = "bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb"`


`TWITTER_API_KEY = "cccccccccccccccccccccccccccc"`

`TWITTER_API_SECRET = "ddddddddddddddddddddddddddddddddddd"`

### <span style="color:#ff5f27;"> 📝 Imports</span>

In [2]:
!pip install -U unicorn-binance-rest-api --quiet
!pip install -U python-dotenv --quiet

In [3]:
import pandas as pd

In [4]:
from functions import *

from dotenv import load_dotenv
load_dotenv()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rkaha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rkaha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\rkaha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

___

## <span style="color:#ff5f27;"> 💽 Loading Data</span>

### <span style='color:#ff5f27'> 📈 Bitcoin Data

In [54]:
df_bitcoin = parse_btc_data(number_of_days_ago=2000)

df_bitcoin = df_bitcoin[(df_bitcoin.date >= '2021-02-05 10:00:00') & (df_bitcoin.date <= '2022-06-04 23:00:00')] 
df_bitcoin.reset_index(drop=True,inplace=True)

df_bitcoin.head(3)

2022-09-29 21:26:29,318 INFO: New instance of unicorn-binance-rest-api_1.5.0-python_3.8.0 on Windows 10 for exchange False started ...
2022-09-29 21:26:29,320 INFO: Initiating `colorama_0.4.5`


,date,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,unix
0,2021-02-06 00:00:00,38289.32,40955.51,38215.94,39186.94,98757.311183,3.922095e+09,2291646,52015.513362,2.065181e+09,1612566000000
1,2021-02-07 00:00:00,39181.01,39700.00,37351.00,38795.69,84363.679763,3.256521e+09,1976357,40764.388959,1.574483e+09,1612652400000
2,2021-02-08 00:00:00,38795.69,46794.45,37988.89,46374.87,138597.536914,5.881537e+09,3230961,72345.891568,3.069314e+09,1612738800000


In [55]:
df_bitcoin_processed = process_btc_data(df_bitcoin)
df_bitcoin_processed.tail(3)

,date,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,...,exp_std_14_days,momentum_14_days,rate_of_change_14_days,strength_index_14_days,std_56_days,exp_mean_56_days,exp_std_56_days,momentum_56_days,rate_of_change_56_days,strength_index_56_days
481,2022-06-02,29805.84,30689.00,29594.55,30452.62,56961.42928,1.711653e+09,1086183,28555.06607,8.581935e+08,...,1679.699391,133.39,4.286187,46.348521,5039.300314,34124.984906,5597.739238,-12991.57,-27.926222,43.640162
482,2022-06-03,30452.63,30699.00,29282.36,29700.21,54067.44727,1.615617e+09,993769,26583.25141,7.943548e+08,...,1576.588412,499.20,0.866529,43.654773,4996.379059,33969.729641,5559.688788,-12551.80,-30.532276,43.052983
483,2022-06-04,29700.21,29988.88,29485.00,29864.04,25617.90113,7.608743e+08,618037,12971.72460,3.853582e+08,...,1472.337144,418.98,-1.419096,44.412343,4925.337066,33825.670351,5514.223362,-12889.93,-29.163058,43.222349


### Older records may come with time=11pm or 9pm, but new ones time is 10pm. Thats because of timezones Daylight saving time. Lets apply this function to make unix column usable.

In [88]:
def get_hours(unix):
    return unix / 3600000 % 24

In [108]:
def fix_unix(unix):
    if get_hours(unix) == 23.0:
        return unix - 3600000
    if get_hours(unix) == 21.0:
        return unix + 3600000
    else:
        return unix

In [96]:
get_hours(1654293600000)

22.0

In [109]:
df_bitcoin_processed.unix = df_bitcoin_processed.unix.apply(fix_unix)

### <span style='color:#ff5f27'> 💭 Tweets Data

In [110]:
# tweets_textblob = pd.read_csv("https://repo.hops.works/dev/davit/bitcoin/tweets_textblob.csv")
tweets_textblob = pd.read_csv("data/tweets_textblob.csv")
tweets_textblob.head(3)

,date,subjectivity,polarity,unix
0,2021-02-05 00:00:00,462.983446,153.870358,1612476000000
1,2021-02-06 00:00:00,945.521424,356.941159,1612562400000
2,2021-02-07 00:00:00,1055.799641,446.821937,1612648800000


In [111]:
tweets_textblob.unix = tweets_textblob.unix.apply(fix_unix)

In [116]:
# tweets_textblob.to_csv("data/tweets_textblob.csv", index=False)

In [114]:
# tweets_vader = pd.read_csv("https://repo.hops.works/dev/davit/bitcoin/tweets_vader.csv")
tweets_vader = pd.read_csv("data/tweets_vader.csv")

tweets_vader.tail(3)

,date,compound,unix
525,2022-07-15 00:00:00,6349.6604,1657832400000
526,2022-07-16 00:00:00,5737.0585,1657918800000
527,2022-07-17 00:00:00,2427.5174,1658005200000


## <span style="color:#ff5f27;"> 🪄 Creating Feature Groups </span>

### <span style='color:#ff5f27'> 📈 Bitcoin Price Feature Group

In [118]:
btc_price_fg = fs.get_or_create_feature_group(
    name='bitcoin_price',
    description='Bitcoin price aggregated for days',
    version=1,
    primary_key=['unix'],
    online_enabled=True,
    event_time=['unix']
)

btc_price_fg.insert(df_bitcoin_processed)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1152/fs/1100/fg/2745


Uploading Dataframe: 0.00% |          | Rows 0/484 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1152/jobs/named/bitcoin_price_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x2592352a700>, None)

In [15]:
btc_price_fg = fs.get_or_create_feature_group(
    name='bitcoin_price',
        version=1)

b = btc_price_fg.read().sort_values(by=["date", "unix"])

2022-09-29 20:57:43,678 INFO: USE `romantest1_featurestore`
2022-09-29 20:57:44,956 INFO: SELECT `fg0`.`date` `date`, `fg0`.`open` `open`, `fg0`.`high` `high`, `fg0`.`low` `low`, `fg0`.`close` `close`, `fg0`.`volume` `volume`, `fg0`.`quote_av` `quote_av`, `fg0`.`trades` `trades`, `fg0`.`tb_base_av` `tb_base_av`, `fg0`.`tb_quote_av` `tb_quote_av`, `fg0`.`unix` `unix`, `fg0`.`mean_7_days` `mean_7_days`, `fg0`.`mean_14_days` `mean_14_days`, `fg0`.`mean_56_days` `mean_56_days`, `fg0`.`signal` `signal`, `fg0`.`std_7_days` `std_7_days`, `fg0`.`exp_mean_7_days` `exp_mean_7_days`, `fg0`.`exp_std_7_days` `exp_std_7_days`, `fg0`.`momentum_7_days` `momentum_7_days`, `fg0`.`rate_of_change_7_days` `rate_of_change_7_days`, `fg0`.`strength_index_7_days` `strength_index_7_days`, `fg0`.`std_14_days` `std_14_days`, `fg0`.`exp_mean_14_days` `exp_mean_14_days`, `fg0`.`exp_std_14_days` `exp_std_14_days`, `fg0`.`momentum_14_days` `momentum_14_days`, `fg0`.`rate_of_change_14_days` `rate_of_change_14_days`, `

In [16]:
tweets_vader_fg = fs.get_or_create_feature_group(
    name='bitcoin_tweets_vader',
    version=1)

v = tweets_vader_fg.read().sort_values(by=["date", "unix"])

2022-09-29 20:58:08,007 INFO: USE `romantest1_featurestore`
2022-09-29 20:58:09,099 INFO: SELECT `fg0`.`date` `date`, `fg0`.`compound` `compound`, `fg0`.`unix` `unix`
FROM `romantest1_featurestore`.`bitcoin_tweets_vader_1` `fg0`


In [13]:
tweets_textblob_fg = fs.get_or_create_feature_group(
    name='bitcoin_tweets_textblob',
    version=1)

t = tweets_textblob_fg.read().sort_values(by=["date", "unix"])

2022-09-29 20:54:42,110 INFO: USE `romantest1_featurestore`
2022-09-29 20:54:43,124 INFO: SELECT `fg0`.`date` `date`, `fg0`.`subjectivity` `subjectivity`, `fg0`.`polarity` `polarity`, `fg0`.`unix` `unix`
FROM `romantest1_featurestore`.`bitcoin_tweets_textblob_1` `fg0`


### <span style='color:#ff5f27'> 💭 Tweets Feature Groups

In [119]:
tweets_textblob_fg = fs.get_or_create_feature_group(
    name='bitcoin_tweets_textblob',
    version=1,
    primary_key=['unix'],
    online_enabled=True,
    event_time=['unix']
)

tweets_textblob_fg.insert(tweets_textblob)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1152/fs/1100/fg/2746


Uploading Dataframe: 0.00% |          | Rows 0/528 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1152/jobs/named/bitcoin_tweets_textblob_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x2592353d070>, None)

In [120]:
tweets_vader_fg = fs.get_or_create_feature_group(
    name='bitcoin_tweets_vader',
    version=1,
    primary_key=['unix'],
    online_enabled=True,
    event_time=['unix']
)

tweets_vader_fg.insert(tweets_vader)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1152/fs/1100/fg/2747


Uploading Dataframe: 0.00% |          | Rows 0/528 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/1152/jobs/named/bitcoin_tweets_vader_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x2592125fcd0>, None)

---